In [0]:
!mkdir train_data

In [2]:
cd train_data

/content/train_data


In [3]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bird.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/circle.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/candle.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cloud.npy


Redirecting output to ‘wget-log’.

Redirecting output to ‘wget-log.1’.

Redirecting output to ‘wget-log.2’.

Redirecting output to ‘wget-log.3’.

Redirecting output to ‘wget-log.4’.

Redirecting output to ‘wget-log.5’.

Redirecting output to ‘wget-log.6’.

Redirecting output to ‘wget-log.7’.

Redirecting output to ‘wget-log.8’.


In [3]:
!ls

apple.npy   book.npy	cloud.npy      wget-log    wget-log.3  wget-log.6
banana.npy  candle.npy	grapes.npy     wget-log.1  wget-log.4  wget-log.7
bird.npy    circle.npy	pineapple.npy  wget-log.2  wget-log.5  wget-log.8


In [5]:
cd ..

/content


In [10]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 3.2MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [0]:
from random import randint
import numpy as np
from os import listdir
from PIL import Image
import tflearn
from tflearn.layers import dropout,conv_2d,max_pool_2d
from tflearn.layers.core import input_data, fully_connected,flatten
from tflearn.layers.estimator import regression

In [2]:
no_classes = 9

# number of samples to take in each class
N = 5000

# some other constants
N_EPOCHS = 10

# data files in the 
files = [i for i in listdir("train_data/") if "npy" in i]
print(files)

['bird.npy', 'grapes.npy', 'circle.npy', 'book.npy', 'candle.npy', 'banana.npy', 'apple.npy', 'cloud.npy', 'pineapple.npy']


In [0]:
def load(dir, reshaped, files):
    "Load .npy or .npz files from disk and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    "Takes a list or a list of lists and returns its normalized form"

    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    "Takes a list or a list of lists and returns its denormalized form"

    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels
  
  
def get_one_hot(targets, nb_classes):
    "one hot encoding"
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [0]:

#second argument is True for reshaping the image to a 28x28 form. A conv net expects this format.
data = load("train_data/", True, files)
 

# limit no of samples in each class to N
data = set_limit(data, N)

# normalize the values
data = list(map(normalize, data))

# define the labels
labels = make_labels(no_classes, N)
labels = get_one_hot(np.array(labels),no_classes)
# print(data[10],labels[10])


In [5]:
model = input_data((28,28,1),name="input")
model = conv_2d(model,32,(3,3),activation='relu')
model = conv_2d(model,64,(3,3),activation='relu')
model = max_pool_2d(model,(2,2))
model = dropout(model,0.25)
model = flatten(model)
model = fully_connected(model,128,activation='relu')
model = dropout(model,0.5)
model = fully_connected(model,no_classes,activation='softmax')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [6]:
model = regression(model, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
# Training
model = tflearn.DNN(model, tensorboard_verbose=3)
model.load('doodle_classifier.tflearn')
model.fit({'input': data}, {'target': labels}, n_epoch=N_EPOCHS,batch_size=32,
              validation_set=0.3, show_metric=True, run_id='doodle_classifier_1.0')
model.save('doodle_classifier_1.0.tflearn')

Training Step: 19699  | total loss: 0.30196 | time: 159.684s
| Adam | epoch: 010 | loss: 0.30196 - acc: 0.9420 -- iter: 31488/31499
Training Step: 19700  | total loss: 0.31970 | time: 162.531s
| Adam | epoch: 010 | loss: 0.31970 - acc: 0.9353 | val_loss: 0.16561 - val_acc: 0.9493 -- iter: 31499/31499
--
INFO:tensorflow:/content/doodle_classifier_1.0.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
!ls

checkpoint
doodle_classifier_1.0.tflearn.data-00000-of-00001
doodle_classifier_1.0.tflearn.index
doodle_classifier_1.0.tflearn.meta
doodle_classifier.tflearn.data-00000-of-00001
doodle_classifier.tflearn.index
doodle_classifier.tflearn.meta
sample_data
train_data


In [0]:
from google.colab import files
files.download('doodle_classifier_1.0.tflearn.data-00000-of-00001') 
files.download('doodle_classifier_1.0.tflearn.meta') 
files.download('doodle_classifier_1.0
               .tflearn.index') 